# DART.fss.or.kr에서 파일 다운로드하기

In [1]:
#json을 읽기 위해서 우선 requests가 필요하고...
import requests
import pandas as pd
#나중에 등장하는 dcm_no의 xpath을 구하기 위한 lxml 
from lxml import html
#RegEx를 나중에 쓰게 된다면 필요한 re
import re

%load_ext autoreload
%autoreload 2

from utils import *

#위에 있는 download_file util 대신 curl을 이용할까 했으나 한국말을 못알아들어서 포기...
#from subprocess import call

API Key를 넣어주세요. [인증키 신청](http://dart.fss.or.kr/dsap001/apikeyManagement.do;jsessionid=Bs7AWiSzD8YmbBx0Zg3WoEixviKFJ7tL2OmeavY5lXpuYNh4MBmNjvvrgldaazhx.dart2_servlet_engine2)은 DART 계정을 만든 후 간단하게 할 수 있습니다

In [2]:
API_KEY = open('api_key.txt','r').read()

## 1. 원하는 회사의 사업보고서 링크 목록을 가져와봅시다. 

#### 1) 회사의 종목코드를 가져오세요. (엑셀 출처: [한국거래소 전자공시 홈페이지](http://kind.krx.co.kr/corpgeneral/corpList.do?method=loadInitPage))

In [3]:
#회사 정보가 들어있는 엑셀을 읽어오되, 종목코드는 int가 아닌 str로 가져와야 합니다 (안 그러면 앞의 0이 지워져서 나오게 돼요)
#엑셀 source: http://kind.krx.co.kr/corpgeneral/corpList.do?method=loadInitPage
company_codes = pd.read_excel('company_codes.xlsx',converters={'종목코드':str})

In [5]:
#회사명 입력란을 만들어요
name_input = input('회사명을 입력해주세요: ')

회사명을 입력해주세요: 피


In [6]:
#입력된 회사명이 없으면 진행이 안돼요
#CAVEAT: CJ같이 단독으로도 회사명이 존재하지만 CJ오쇼핑 같이 이게 포함된 회사명이 있는 경우, 찾아주지는 못합니다
while len(company_codes[company_codes.회사명 ==  name_input]) == 0:
    print('해당 이름의 회사명이 존재하지 않습니다. \n아래 회사명 중 하나를 찾으시나요? 다시 입력해주세요.\n')
    for row in company_codes.회사명:
        if row.find(name_input) != -1:
            print(row)
    name_input = input()
code = company_codes[company_codes.회사명 == name_input].종목코드.iloc[0]
print(name_input+" 종목코드: "+code)

해당 이름의 회사명이 존재하지 않습니다. 
아래 회사명 중 하나를 찾으시나요? 다시 입력해주세요.

동양피스톤
에스엔피월드
케이피에스
이엘피
피씨엘
피엔아이시스템
피시피아비아이티
세화피앤씨
피엔에이치테크
제이에스피브이
피앤씨테크
한솔씨앤피
럭스피아
유진에이씨피씨스팩2호
유디피
피노텍
슈피겐코리아
영우디에스피
엘피케이
에스엔피제네틱스
피엠디아카데미
피엔티
피앤이솔루션
티피씨글로벌
피제이메탈
디에이치피코리아
인피니트헬스케어
동양피엔에프
아주캐피탈
상신이디피
동양이엔피
케이에스피
디에이피
케이피엠테크
에스피지
대진디엠피
케이피티
피씨디렉트
피앤텔
피에스엠씨
금화피에스시
엔피케이
디피씨
피델릭스
피에스케이
메이슨캐피탈
대호피앤씨
케이씨피드
케이피에프
한국캐피탈
지엠피
피제이전자
큐캐피탈
피에스텍
한솔피엔에스
에이엔피
조광피혁
엔피씨
피혁
해당 이름의 회사명이 존재하지 않습니다. 
아래 회사명 중 하나를 찾으시나요? 다시 입력해주세요.

조광피혁
조광
해당 이름의 회사명이 존재하지 않습니다. 
아래 회사명 중 하나를 찾으시나요? 다시 입력해주세요.

조광ILI
조광피혁
조광페인트
조광페인트
조광페인트 종목코드: 004910


#### 2) 보고서 목록 URL을 생성하세요

In [7]:
#t시작날짜는 최초의 기업이 상장한 날짜인 1956년 3월보다 이전으로 잡았습니다
start_date = '19560101'
#Document type: 사업보고서
bsn_tp = 'A001'

In [8]:
url = "http://dart.fss.or.kr/api/search.json?auth="+API_KEY+"&crp_cd="+code+"&start_dt="+start_date+"&bsn_tp="+bsn_tp+"&fin_rpt=Y&page_set=100"

#### 3) 개별 보고서 URL을 생성하세요

In [9]:
#json 값을 추출합시다
a = requests.get(url).json()

In [10]:
#각 사업보고서 당 리스트가 제대로 생성되는 지 봅시다. 하나도 없으면 코드 돌려봤자 아무것도 다운 안 됨
urldict = {}
for row in a['list']:
    url2 = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo="
    name = row['rpt_nm']
    #[기재정정] [첨부추가] [첨부정정] 등 앞에 붙은 것을 제거해봅시다
    if name.find('[') != -1:
        name = name.split(']')[1]
    urldict[name] = url2+row['rcp_no']
    print(name+": "+url2+row['rcp_no'])

사업보고서 (2016.12): http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170329000180
사업보고서 (2015.12): http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160329000196
사업보고서 (2014.12): http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20150330000683
사업보고서 (2013.12): http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20140327000156
사업보고서 (2012.12): http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20130328000171
사업보고서 (2011.12): http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20120514000508
사업보고서 (2010.12): http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20110331000297
사업보고서 (2009.12): http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20100331000275
사업보고서 (2008.12): http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20090330000266
사업보고서 (2007.12): http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20080328000029
사업보고서 (2006.12): http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20070328000563
사업보고서 (2005.12): http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20060504000097
사업보고서 (2004.12): http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20050331000004
사업보고서 (2003.

## 2. 각 사업보고서의 첨부파일 리스트를 확인하고 다운로드합시다

In [11]:
#카운터 선정
n=1

for key, value in urldict.items(): 
    #dcm_no 값을 알아야 다운로드 링크에 접근할 수 있는데, 알 방법이 링크에서 바로 가져오는 방법밖에 없으므로 xpath을 활용해서 알아봅시다
    test = requests.get(value)
    tree = html.fromstring(test.content)
    testpath = tree.xpath('//*[@id="north"]/div[2]/ul/li[1]/a/@onclick')[0]
    dcm_no = dcm_no = testpath.split(", '")[1].split("')")[0]
    
    #다운로드를 위한 url은 보고서 url과 차이점이 몇 가지 있는데, replace를 통해 추가할 수 있어요
    download_url = value.replace('dsaf001','pdf/download').replace('rcpNo','rcp_no')+"&dcm_no="+dcm_no
    print(key+" "+download_url+" 다운 중... "+str(n)+" out of "+str(len(urldict)))
    
    #dcm_no를 구했던 것과 같은 방법으로 첨부파일 다운로드 url을 추출합니다
    dtest = requests.get(download_url)
    dtree = html.fromstring(dtest.text)
    
    #각 보고서 당 복수의 첨부파일이 존재하는데, 첨부파일 이름과 함께 저장하기 위해 downloadpath라는 dict를 사용했습니다
    downloadpath={}
    keys = dtree.xpath('/html/body/div/div/table/tr/td[1]/text()')
    key_links = dtree.xpath('/html/body/div/div/table/tr/td/a/@href')
    for key2, link in zip(keys, key_links):
        l = "http://dart.fss.or.kr"+link
        k = key2.replace(")","")
        downloadpath[k] = l
    #print(downloadpath)
        
    #utils에 있는 download_file을 이요해 디렉토리를 만들고 그 안에다가 파일을 집어넣습니다
    for key2, link in downloadpath.items():
        download_file(link,filename=key2,directory="dart_"+name_input+"/"+key)
        #try:
            #os.mkdir(key)
        #except:
            #pass
        #call(['curl',link,'-o',key+'/'+key2])
    n+=1

사업보고서 (2016.12) http://dart.fss.or.kr/pdf/download/main.do?rcp_no=20170329000180&dcm_no=5511995 다운 중... 1 out of 19
사업보고서 (2015.12) http://dart.fss.or.kr/pdf/download/main.do?rcp_no=20160329000196&dcm_no=5006277 다운 중... 2 out of 19
사업보고서 (2014.12) http://dart.fss.or.kr/pdf/download/main.do?rcp_no=20150330000683&dcm_no=4541050 다운 중... 3 out of 19
사업보고서 (2013.12) http://dart.fss.or.kr/pdf/download/main.do?rcp_no=20140327000156&dcm_no=4093767 다운 중... 4 out of 19
사업보고서 (2012.12) http://dart.fss.or.kr/pdf/download/main.do?rcp_no=20130328000171&dcm_no=3686972 다운 중... 5 out of 19
사업보고서 (2011.12) http://dart.fss.or.kr/pdf/download/main.do?rcp_no=20120514000508&dcm_no=3413538 다운 중... 6 out of 19
사업보고서 (2010.12) http://dart.fss.or.kr/pdf/download/main.do?rcp_no=20110331000297&dcm_no=2956560 다운 중... 7 out of 19
사업보고서 (2009.12) http://dart.fss.or.kr/pdf/download/main.do?rcp_no=20100331000275&dcm_no=2584703 다운 중... 8 out of 19
사업보고서 (2008.12) http://dart.fss.or.kr/pdf/download/main.do?rcp_no=200903